In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

## Auto

In [22]:
start_date = '2022-09-01'
end_date = '2022-09-30'

In [23]:
auto = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(auto_lifetime_last_ride_date) 
    auto_lifetime_last_ride_date,
    auto_lifetime_first_ride_date,
    --MAX(auto_lifetime_rides) 
    auto_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (auto_lifetime_rides > 0 or auto_lifetime_rides is not null or auto_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN auto_lifetime_rides IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 THEN '1-4'
    WHEN (auto_lifetime_rides BETWEEN 5 AND 20) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '5-20'
    WHEN (auto_lifetime_rides BETWEEN 21 AND 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '21-50'
    WHEN (auto_lifetime_rides > 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '50+'
    WHEN auto_lifetime_rides >= 5 THEN '>=5'
    ELSE 'LTR=0'
    END ltr_auto,
    CASE 
    WHEN auto_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '1. LAST_30_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '2. 31_90_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '3. 91_180_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '4. 181_365_DAYS'
    WHEN auto_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '5. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_auto
FROM base_month_customer
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Auto'
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_auto,'NEW') ltr_auto,
        COALESCE(b.recency_auto, 'NEW') recency_auto,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_auto,
        case 
        when ltr_auto = '1-4' then 1
        when ltr_auto = '5-20' then 2
        when ltr_auto = '21-50' then 3
        when ltr_auto = '50+' then 4
        when ltr_auto = '>=5' then 5
        when ltr_auto = 'LTR=0' then 6
        when ltr_auto = 'NEW' then 7
        end row_order,
        recency_auto,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_auto,
        a.recency_auto,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_auto,
        '' recency_auto,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_auto,recency_auto,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [24]:
df_auto = pd.read_sql(auto, conn)

In [25]:
df_auto_sep = df_auto
df_auto_sep.head()

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,1488718,10.71,1060357,71.23,971383,65.25,611857,...,28.54,69.43,14.33,933144,11.72,2.20,3873641.0,125856897.0,3.077814,4.151172
1,2022-09-01,1-4,2. 31_90_DAYS,1852611,13.32,967506,52.22,848577,45.80,404326,...,12.76,58.47,7.98,394753,4.96,1.67,1831934.0,54728029.0,3.347341,4.640710
2,2022-09-01,1-4,3. 91_180_DAYS,1563440,11.24,589425,37.70,507233,32.44,194973,...,6.92,55.46,3.65,167692,2.11,1.55,842248.0,23406556.0,3.598342,5.022589
3,2022-09-01,1-4,4. 181_365_DAYS,1332403,9.58,350423,26.30,304663,22.87,97864,...,3.98,54.13,1.79,81260,1.02,1.53,461379.0,11229863.0,4.108501,5.677812
4,2022-09-01,1-4,5. 365_ABOVE,647846,4.66,134706,20.79,117264,18.10,31529,...,2.55,52.37,0.56,25067,0.31,1.52,152579.0,3481943.0,4.382007,6.086847


In [27]:
df_auto_nov

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,1-4,1. LAST_30_DAYS,1327552,8.17,948658,71.46,863136,65.02,524933,...,26.40,66.77,12.41,739939,9.94,2.11,1806696.0,97322462.0,1.856402,2.441682
1,2022-11-01,1-4,2. 31_90_DAYS,1943050,11.96,976288,50.25,853745,43.94,396681,...,11.54,56.51,7.94,367451,4.94,1.64,1066526.0,49593223.0,2.150548,2.902499
2,2022-11-01,1-4,3. 91_180_DAYS,2025002,12.47,743120,36.70,633413,31.28,236729,...,6.12,52.31,4.38,188135,2.53,1.52,528617.0,25885582.0,2.042129,2.809775
3,2022-11-01,1-4,4. 181_365_DAYS,1703400,10.49,442450,25.97,378422,22.22,115685,...,3.50,51.54,2.11,88316,1.19,1.48,289012.0,12090954.0,2.390316,3.272476
4,2022-11-01,1-4,5. 365_ABOVE,964260,5.94,185610,19.25,158382,16.43,40007,...,2.07,49.83,0.71,29106,0.39,1.46,124239.0,4011536.0,3.097043,4.268501
5,2022-11-01,5-20,1. LAST_30_DAYS,1125295,6.93,984211,87.46,946163,84.08,747121,...,51.38,77.38,20.47,1703746,22.89,2.95,3003395.0,216819747.0,1.385204,1.762819
6,2022-11-01,21-50,1. LAST_30_DAYS,343603,2.12,323243,94.07,317488,92.40,289579,...,73.02,86.65,8.88,1199068,16.11,4.78,1921922.0,145386457.0,1.321940,1.602847
7,2022-11-01,50+,1. LAST_30_DAYS,144279,0.89,138988,96.33,137646,95.40,133083,...,86.22,93.47,4.40,1179697,15.85,9.48,2075370.0,129184303.0,1.606519,1.759240
8,2022-11-01,>=5,2. 31_90_DAYS,1001031,6.16,686698,68.60,635777,63.51,405101,...,26.34,65.08,9.33,533527,7.17,2.02,1031215.0,69798345.0,1.477420,1.932826
9,2022-11-01,>=5,3. 91_180_DAYS,605406,3.73,287354,47.46,256026,42.29,127665,...,12.22,57.96,2.62,126642,1.70,1.71,233289.0,17009865.0,1.371492,1.842114


In [28]:
df_auto_oct

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,1-4,1. LAST_30_DAYS,1444300,9.58,1045530,72.39,969386,67.12,626504,...,28.28,65.19,13.69,848711,11.22,2.08,2178986.0,1.154492e+08,1.887397,2.567406
1,2022-10-01,1-4,2. 31_90_DAYS,1922552,12.76,1017718,52.94,907810,47.22,457071,...,12.78,53.74,8.23,395441,5.23,1.61,1194690.0,5.548502e+07,2.153176,3.021159
2,2022-10-01,1-4,3. 91_180_DAYS,1809109,12.00,695271,38.43,606504,33.53,252944,...,6.86,49.07,4.16,186379,2.46,1.50,559040.0,2.633804e+07,2.122557,2.999480
3,2022-10-01,1-4,4. 181_365_DAYS,1466169,9.73,397987,27.14,349503,23.84,120577,...,3.95,47.99,1.94,85203,1.13,1.47,282659.0,1.200501e+07,2.354509,3.317477
4,2022-10-01,1-4,5. 365_ABOVE,823296,5.46,171769,20.86,151121,18.36,43710,...,2.40,45.13,0.66,28851,0.38,1.46,115845.0,4.047786e+06,2.861935,4.015285
5,2022-10-01,5-20,1. LAST_30_DAYS,1117857,7.42,996446,89.14,967911,86.59,795731,...,55.10,77.41,20.64,1800814,23.80,2.92,3538371.0,2.390728e+08,1.480039,1.964873
6,2022-10-01,21-50,1. LAST_30_DAYS,321523,2.13,304080,94.57,299809,93.25,277873,...,75.70,87.60,8.16,1158497,15.31,4.76,2013445.0,1.473235e+08,1.366683,1.737980
7,2022-10-01,50+,1. LAST_30_DAYS,127682,0.85,122999,96.33,122009,95.56,118437,...,87.32,94.13,3.74,1051462,13.90,9.43,2189576.0,1.238669e+08,1.767685,2.082411
8,2022-10-01,>=5,2. 31_90_DAYS,918932,6.10,643960,70.08,603532,65.68,403217,...,27.78,63.32,8.55,499934,6.61,1.96,1059047.0,6.862301e+07,1.543283,2.118374
9,2022-10-01,>=5,3. 91_180_DAYS,496008,3.29,244947,49.38,222308,44.82,119850,...,13.47,55.74,2.24,113093,1.49,1.69,250292.0,1.568910e+07,1.595324,2.213152


In [29]:
df_auto_sep

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,1488718,10.71,1060357,71.23,971383,65.25,611857,...,28.54,69.43,14.33,933144,11.72,2.20,3873641.0,1.258569e+08,3.077814,4.151172
1,2022-09-01,1-4,2. 31_90_DAYS,1852611,13.32,967506,52.22,848577,45.80,404326,...,12.76,58.47,7.98,394753,4.96,1.67,1831934.0,5.472803e+07,3.347341,4.640710
2,2022-09-01,1-4,3. 91_180_DAYS,1563440,11.24,589425,37.70,507233,32.44,194973,...,6.92,55.46,3.65,167692,2.11,1.55,842248.0,2.340656e+07,3.598342,5.022589
3,2022-09-01,1-4,4. 181_365_DAYS,1332403,9.58,350423,26.30,304663,22.87,97864,...,3.98,54.13,1.79,81260,1.02,1.53,461379.0,1.122986e+07,4.108501,5.677812
4,2022-09-01,1-4,5. 365_ABOVE,647846,4.66,134706,20.79,117264,18.10,31529,...,2.55,52.37,0.56,25067,0.31,1.52,152579.0,3.481943e+06,4.382007,6.086847
5,2022-09-01,5-20,1. LAST_30_DAYS,1106217,7.96,976990,88.32,941291,85.09,755226,...,54.53,79.87,20.35,1900903,23.88,3.15,5175264.0,2.506961e+08,2.064358,2.722529
6,2022-09-01,21-50,1. LAST_30_DAYS,294135,2.12,277171,94.23,272309,92.58,249756,...,75.56,88.99,7.50,1184266,14.88,5.33,2726360.0,1.507796e+08,1.808175,2.302152
7,2022-09-01,50+,1. LAST_30_DAYS,109318,0.79,105051,96.10,103960,95.10,100572,...,87.17,94.75,3.22,1026037,12.89,10.77,2770321.0,1.245161e+08,2.224870,2.700021
8,2022-09-01,>=5,2. 31_90_DAYS,795785,5.72,543260,68.27,502118,63.10,320033,...,27.01,67.17,7.25,436259,5.48,2.03,1508658.0,6.013162e+07,2.508926,3.458170
9,2022-09-01,>=5,3. 91_180_DAYS,401938,2.89,193512,48.14,173236,43.10,86673,...,13.14,60.94,1.78,91726,1.15,1.74,359675.0,1.283140e+07,2.803084,3.921189


In [30]:
frames = [df_auto_nov, df_auto_oct, df_auto_sep]

In [31]:
nov_oct_sep = pd.concat(frames)

In [32]:
nov_oct_sep

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,1-4,1. LAST_30_DAYS,1327552,8.17,948658,71.46,863136,65.02,524933,...,26.40,66.77,12.41,739939,9.94,2.11,1806696.0,9.732246e+07,1.856402,2.441682
1,2022-11-01,1-4,2. 31_90_DAYS,1943050,11.96,976288,50.25,853745,43.94,396681,...,11.54,56.51,7.94,367451,4.94,1.64,1066526.0,4.959322e+07,2.150548,2.902499
2,2022-11-01,1-4,3. 91_180_DAYS,2025002,12.47,743120,36.70,633413,31.28,236729,...,6.12,52.31,4.38,188135,2.53,1.52,528617.0,2.588558e+07,2.042129,2.809775
3,2022-11-01,1-4,4. 181_365_DAYS,1703400,10.49,442450,25.97,378422,22.22,115685,...,3.50,51.54,2.11,88316,1.19,1.48,289012.0,1.209095e+07,2.390316,3.272476
4,2022-11-01,1-4,5. 365_ABOVE,964260,5.94,185610,19.25,158382,16.43,40007,...,2.07,49.83,0.71,29106,0.39,1.46,124239.0,4.011536e+06,3.097043,4.268501
5,2022-11-01,5-20,1. LAST_30_DAYS,1125295,6.93,984211,87.46,946163,84.08,747121,...,51.38,77.38,20.47,1703746,22.89,2.95,3003395.0,2.168197e+08,1.385204,1.762819
6,2022-11-01,21-50,1. LAST_30_DAYS,343603,2.12,323243,94.07,317488,92.40,289579,...,73.02,86.65,8.88,1199068,16.11,4.78,1921922.0,1.453865e+08,1.321940,1.602847
7,2022-11-01,50+,1. LAST_30_DAYS,144279,0.89,138988,96.33,137646,95.40,133083,...,86.22,93.47,4.40,1179697,15.85,9.48,2075370.0,1.291843e+08,1.606519,1.759240
8,2022-11-01,>=5,2. 31_90_DAYS,1001031,6.16,686698,68.60,635777,63.51,405101,...,26.34,65.08,9.33,533527,7.17,2.02,1031215.0,6.979834e+07,1.477420,1.932826
9,2022-11-01,>=5,3. 91_180_DAYS,605406,3.73,287354,47.46,256026,42.29,127665,...,12.22,57.96,2.62,126642,1.70,1.71,233289.0,1.700986e+07,1.371492,1.842114


In [34]:
nov_oct_sep.to_csv("auto_nov_oct_sep.csv", index = False)